In [1]:
import sys
sys.path.append('C:\\PYTHONprojects\\en')
sys.path.append('C:\\PYTHONprojects')
sys.path.append('C:\\PYTHONprojects\\utilities')
import morePVs_output as opm
import os

import en_utilities as um
import pandas as pd
import shutil
project='EN2_bat2'
base_path='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_4\\studies'
i_path =os.path.join(base_path,project,'inputs')

## `bat_en2` is bau / en / en_pv comparison version with
*  New `battery_strategies` with earlier evening discharge:
`ed1_17
ed2_17
ed3_17
ed1_1730
ed2_1730
ed3_1730`

* TOU retail tariffs at parent meter
* range of parent tariffs, including FiTs
* Range of PV
* Range of kWh BESS
* Range of $ /kWh BESS

`en` and `en_pv` only


In [2]:
# Set up all variables

sites=['A','B','C','D','E','F','G','H','I','J']
pvs = {'A':['max'],
      'B':['max'],
      'C':['max'],
      'D':['max'],
      'E':['max'],
      'F':['max','0_5kw','1_0kw'],
      'G':['max','0_5kw','1_0kw','1_5kw'],
      'H':['max','0_5kw','1_0kw','1_5kw','2_0kw','2_5kw'],
      'I':['max','0_5kw','1_0kw'],
      'J':['max','0_5kw','1_0kw','1_5kw','2_0kw','2_5kw']
      }
pv_base = 'site_'
btm_i = ['btm_i_u','btm_i_c']
btm_s = ['btm_s_c', 'btm_s_u']
btm_p = ['btm_p_c', 'btm_p_u']
btms_capexes= [ 'upfront_1']
btmp_capexes = ['ppa_1']
en_capexes= ['capex_low','capex_med','capex_high']
net_tariffs = {'A':'EA310',
      'B':'EA310',
      'C':'EA305',
      'D':'EA310',
      'E':'EA310',
      'F':'EA305',
      'G':'EA305',
      'H':'EA305',
      'I':'EA305',
      'J':'EA305'
      }
parent_retails =   [ 'TOU9_FIT8', 'TOU9_FIT12','TOU9',
                      'TOU12_FIT8', 'TOU12_FIT12', 'TOU12']
                       # NEED TO ADD THESE: 'TOU15_FIT8', 'TOU15_FIT12', 'TOU15']
parents = {'EA310': ['EA310_' + t for t in parent_retails],
           'EA305': ['EA305_' + t for t in parent_retails]}
         
                
terms =[5,10,15,20,25]
rates = [0.06]
btmi_tariffs = ['EASO_TOU_15pc', 'EASO_TOU_15pc_FIT8','EASO_TOU_15pc_FIT12']
btms_tariffs = ['EASO_TOU_15pc', 'EASO_TOU_15pc_FIT8','EASO_TOU_15pc_FIT12']
               
btmp_tariffs =['SIT_15_ppa1','SIT_15_FIT8_ppa1','SIT_15_FIT12_ppa1',
                'SIT_15_ppa2', 'SIT_15_FIT8_ppa2','SIT_15_FIT12_ppa2']
bau_tariff= btmi_tariffs[0]

tags = ['a208_f12_cp34',
 'a161_f7_cp38',
 'a138_f9_cp44',
 'a104_f8_cp57',
 'a52_f3_cp26',
 'a48_f4_cp09',
 'a44_f4_cp17',
 'a34_f4_cp33',
 'a26_f4_cp43',
 'a20_f5_cp36']

site_tags = {'A': 'a208_f12_cp34',
 'B': 'a104_f8_cp57',
 'C': 'a34_f4_cp33',
 'D': 'a138_f9_cp44',
 'E': 'a161_f7_cp38',
 'F': 'a20_f5_cp36',
 'G': 'a44_f4_cp17',
 'H': 'a52_f3_cp26',
 'I': 'a48_f4_cp09',
 'J': 'a26_f4_cp43'}

num_occs = {'A': 208,
             'B': 104,
             'C': 34,
             'D': 138,
             'E': 161,
             'F': 20,
             'G': 44,
             'H': 52,
             'I': 48,
             'J': 26}

cp_ratio = {'A': .34,
             'B': .57,
             'C': .33,
             'D': .44,
             'E': .38,
             'F': .36,
             'G': .17,
             'H': .26,
             'I': .09,
             'J': .43} 


central_battery_capacities= dict(zip(sites, [[n*num_occs[site] for n in [1,2,3,4]] for site in sites]))              
cp_battery_capacities =  dict(zip(sites, [[n * num_occs[site] * cp_ratio[site] for n in [1,2,3,4]] for site in sites]))     
unit_battery_capacities =  dict(zip(sites, [[n  * (1 - cp_ratio[site]) for n in [1,2,3,4]] for site in sites]))  

battery_costs = [250,500,750,1000]
battery_strategies = ['ed1_s_17','ed2_s_17','ed1_s_18','ed2_s_18']


In [ ]:
# -----------------------------------------------------------------------
# Energy analysis: SS, SC and logging timeseries for peak demand analysis
# -----------------------------------------------------------------------
studybase = 'bat_energy3'
# en, en_pv with central battery
# btm_i_c and bau_bat (?) with individual batteries
# All batteries are scaleable
central_battery_id = 'pw_scale'
cp_battery_id = 'pw_scale'
all_battery_id = 'pw_scale'

# Only interested in energy parameters, so financials are dummies

# SIMPLIFICATIONs:

en_capexes = ['capex_med']
net_tariff = 'EA305'
parent = 'EA305_TOU9'
tariff = 'EASO_TOU9'
terms=[20]
battery_costs = ['1000']
#sites =['J']  
sites =['H','I','G','J','F' ]

# Seperate `.csv` for each site:
for site in sites:
    net_tariff = net_tariffs[site]
    fn = 'study_' + studybase + '_' + site  +'.csv'
    file = os.path.join(i_path,fn)
    df = pd.DataFrame()
    df.index.name='scenario'
    lf = 'sh_t_all_site_'+site
    
    s=-1 # To start at 0 for bau
    
    arr ='bau'
    for term in terms:
        for rate in rates:
            s += 1
            df.loc[s,'pv_filename'] = ''
            df.loc[s,'load_folder'] = lf
            df.loc[s,'arrangement'] = arr
            df.loc[s,'pv_cap_id'] = ''
            df.loc[s,'cp'] = bau_tariff
            df.loc[s,'all_residents'] = bau_tariff
            df.loc[s,'parent'] = 'TIDNULL'
            df.loc[s,'network_tariff'] = net_tariff
            df.loc[s,'en_capex_id'] = ''
            df.loc[s,'a_term'] = term
            df.loc[s,'a_rate'] = rate
            df.loc[s,'output_types'] = 'log_timeseries_brief'
    
                
    arr ='en'
    tariff = 'TIDNULL'

    for capex in en_capexes:
        for term in terms:
            for rate in rates:
                for battery_capex_per_kWh in battery_costs:
                    for central_battery_capacity_kWh in central_battery_capacities[site]:
                        for central_battery_strategy in battery_strategies:
                            s += 1
                            df.loc[s,'pv_filename'] = ''
                            df.loc[s,'load_folder'] = lf
                            df.loc[s,'arrangement'] = arr
                            df.loc[s,'pv_cap_id'] = ''
                            df.loc[s,'cp'] = tariff
                            df.loc[s,'all_residents'] = tariff
                            df.loc[s,'parent'] = parent
                            df.loc[s,'network_tariff'] = net_tariff
                            df.loc[s,'en_capex_id'] = capex
                            df.loc[s,'a_term'] = term
                            df.loc[s,'a_rate'] = rate
                            df.loc[s,'central_battery_id'] = central_battery_id
                            df.loc[s,'central_battery_strategy'] = central_battery_strategy
                            df.loc[s,'central_battery_capacity_kWh'] = central_battery_capacity_kWh                               
                            df.loc[s,'battery_capex_per_kWh'] = battery_capex_per_kWh
                            df.loc[s,'output_types'] = 'log_timeseries_brief'

                                
    arr ='en_pv' # with no battery
    tariff = 'TIDNULL'
    for pv in pvs[site]:
               
        for capex in en_capexes:
            for term in terms:
                for rate in rates:
                    s += 1
                    df.loc[s,'pv_filename'] = pv_base + site+ '_' + pv +'.csv'
                    df.loc[s,'load_folder'] = lf
                    df.loc[s,'arrangement'] = arr
                    df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
                    df.loc[s,'cp'] = tariff
                    df.loc[s,'all_residents'] = tariff
                    df.loc[s,'parent'] = parent
                    df.loc[s,'network_tariff'] = net_tariff
                    df.loc[s,'en_capex_id'] = capex
                    df.loc[s,'a_term'] = term
                    df.loc[s,'a_rate'] = rate
                    df.loc[s,'central_battery_id'] = ''
                    df.loc[s,'central_battery_strategy'] = ''
                    df.loc[s,'output_types'] = 'log_timeseries_brief'
                               
                    
    arr ='en_pv'
    tariff = 'TIDNULL'
    for pv in pvs[site]:           
        for capex in en_capexes:
            for term in terms:
                for rate in rates:
                    for battery_capex_per_kWh in battery_costs:
                        for central_battery_capacity_kWh in central_battery_capacities[site]:
                            for central_battery_strategy in battery_strategies:
                                s += 1
                                df.loc[s,'pv_filename'] = pv_base + site+ '_' + pv +'.csv'
                                df.loc[s,'load_folder'] = lf
                                df.loc[s,'arrangement'] = arr
                                df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
                                df.loc[s,'cp'] = tariff
                                df.loc[s,'all_residents'] = tariff
                                df.loc[s,'parent'] = parent
                                df.loc[s,'network_tariff'] = net_tariff
                                df.loc[s,'en_capex_id'] = capex
                                df.loc[s,'a_term'] = term
                                df.loc[s,'a_rate'] = rate
                                df.loc[s,'central_battery_id'] = central_battery_id
                                df.loc[s,'central_battery_strategy'] = central_battery_strategy
                                df.loc[s,'central_battery_capacity_kWh'] = central_battery_capacity_kWh          
                                df.loc[s,'battery_capex_per_kWh'] = battery_capex_per_kWh
                                df.loc[s,'output_types'] = 'log_timeseries_brief'
               
                        
#     arr = 'btm_i_c'
#     parent ='TIDNULL'
#     tariff = 'TIDNULL'
#     capex ='' 
#     cp_battery_id = 'pw_scale'
#     all_battery_id = 'pw_scale'
#     for pv in pvs[site]:            
#             for term in terms:
#                 for rate in rates:
#                     for battery_capex_per_kWh in battery_costs:
#                         for all_battery_capacity_kWh in unit_battery_capacities[site]:
#                             cp_battery_capacity_kWh = cp_battery_capacities[site][
#                                 unit_battery_capacities[site].index(all_battery_capacity_kWh)]
#                             print(site, all_battery_capacity_kWh,cp_battery_capacity_kWh )
#                             for all_battery_strategy in battery_strategies:
#                                 cp_battery_strategy = all_battery_strategy
#                                 s += 1
#                                 df.loc[s,'pv_filename'] = pv_base + site + '_' + pv +'.csv'
#                                 df.loc[s,'load_folder'] = lf
#                                 df.loc[s,'arrangement'] = arr
#                                 df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv + '_i'  # NB different capex for `btm_i`
#                                 df.loc[s,'cp'] = tariff
#                                 df.loc[s,'all_residents'] = tariff
#                                 df.loc[s,'parent'] = parent
#                                 df.loc[s,'network_tariff'] = net_tariff
#                                 df.loc[s,'en_capex_id'] = capex
#                                 df.loc[s,'a_term'] = term
#                                 df.loc[s,'a_rate'] = rate                    
#                                 df.loc[s,'cp_battery_id'] = cp_battery_id
#                                 df.loc[s,'cp_battery_strategy'] = cp_battery_strategy
#                                 df.loc[s,'cp_battery_capacity_kWh'] = cp_battery_capacity_kWh    
#                                 df.loc[s,'all_battery_id'] = all_battery_id
#                                 df.loc[s,'all_battery_strategy'] = all_battery_strategy
#                                 df.loc[s,'all_battery_capacity_kWh'] = all_battery_capacity_kWh    
#                                 df.loc[s,'battery_capex_per_kWh'] = battery_capex_per_kWh
#                                 df.loc[s,'output_types'] = 'log_timeseries_brief'

#     arr= 'cp_only'
#     parent ='TIDNULL'
#     for pv in pvs[site]:     
#         for tariff in btmi_tariffs:
#             for term in terms:
#                 for rate in rates:
#                     s += 1
#                     df.loc[s,'pv_filename'] = pv_base + site+ '_' + pv +'.csv'
#                     df.loc[s,'load_folder'] = lf
#                     df.loc[s,'arrangement'] = arr
#                     df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
#                     df.loc[s,'cp'] = tariff
#                     df.loc[s,'all_residents'] = tariff
#                     df.loc[s,'parent'] = 'TIDNULL'
#                     df.loc[s,'network_tariff'] = net_tariff
#                     df.loc[s,'en_capex_id'] = ''
#                     df.loc[s,'a_term'] = term
#                     df.loc[s,'a_rate'] = rate


    um.df_to_csv(df,file)
    print ("****Saved ", file)
print("****completed****")



In [ ]:
# # ------------------
# # financial analysis
# # ------------------
# studybase = 'bat_finance1'

# # All batteries are scaleable:
# # ---------------------------
# central_battery_id = 'pw_scale'

# # SIMPLIFICATIONs:
# sites =['J', 'I', 'H', 'G','F']
# en_capexes = ['capex_med']
# terms=[10,20]
# battery_strategies= ['evening_discharge_1']


# # Separate `.csv` for each site:
# for site in sites:
#     net_tariff = net_tariffs[site]
#     fn = 'study_' + studybase + '_' + site  +'.csv'
#     file = os.path.join(i_path,fn)
#     df = pd.DataFrame()
#     df.index.name='scenario'
#     lf = 'sh_t_all_site_'+site
    
#     s=-1 # To start at 0 for bau
    
#     arr ='bau'
#     for term in terms:
#         for rate in rates:
#             s += 1
#             df.loc[s,'pv_filename'] = ''
#             df.loc[s,'load_folder'] = lf
#             df.loc[s,'arrangement'] = arr
#             df.loc[s,'pv_cap_id'] = ''
#             df.loc[s,'cp'] = bau_tariff
#             df.loc[s,'all_residents'] = bau_tariff
#             df.loc[s,'parent'] = 'TIDNULL'
#             df.loc[s,'network_tariff'] = net_tariff
#             df.loc[s,'en_capex_id'] = ''
#             df.loc[s,'a_term'] = term
#             df.loc[s,'a_rate'] = rate
    
                
#     arr ='en'
#     tariff = 'TIDNULL'    
#     for parent in parents[net_tariff]:
#         for capex in en_capexes:
#             for term in terms:
#                 for rate in rates:
#                     for battery_capex_per_kWh in battery_costs:
#                         for central_battery_capacity_kWh in central_battery_capacities[site]:
#                             for central_battery_strategy in battery_strategies:
#                                 s += 1
#                                 df.loc[s,'pv_filename'] = ''
#                                 df.loc[s,'load_folder'] = lf
#                                 df.loc[s,'arrangement'] = arr
#                                 df.loc[s,'pv_cap_id'] = ''
#                                 df.loc[s,'cp'] = tariff
#                                 df.loc[s,'all_residents'] = tariff
#                                 df.loc[s,'parent'] = parent
#                                 df.loc[s,'network_tariff'] = net_tariff
#                                 df.loc[s,'en_capex_id'] = capex
#                                 df.loc[s,'a_term'] = term
#                                 df.loc[s,'a_rate'] = rate
#                                 df.loc[s,'central_battery_id'] = central_battery_id
#                                 df.loc[s,'central_battery_strategy'] = central_battery_strategy
#                                 df.loc[s,'central_battery_capacity_kWh'] = central_battery_capacity_kWh
#                                 df.loc[s,'battery_capex_per_kWh'] = battery_capex_per_kWh
                                
#     arr ='en_pv' # with no battery
#     tariff = 'TIDNULL'
#     for pv in pvs[site]:
#         for parent in parents[net_tariff]:           
#             for capex in en_capexes:
#                 for term in terms:
#                     for rate in rates:
#                         s += 1
#                         df.loc[s,'pv_filename'] = pv_base + site+ '_' + pv +'.csv'
#                         df.loc[s,'load_folder'] = lf
#                         df.loc[s,'arrangement'] = arr
#                         df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
#                         df.loc[s,'cp'] = tariff
#                         df.loc[s,'all_residents'] = tariff
#                         df.loc[s,'parent'] = parent
#                         df.loc[s,'network_tariff'] = net_tariff
#                         df.loc[s,'en_capex_id'] = capex
#                         df.loc[s,'a_term'] = term
#                         df.loc[s,'a_rate'] = rate
#                         df.loc[s,'central_battery_id'] = ''
#                         df.loc[s,'central_battery_strategy'] = ''
                               
                    
#     arr ='en_pv'
#     tariff = 'TIDNULL'
#     for pv in pvs[site]:
#         for parent in parents[net_tariff]:           
#             for capex in en_capexes:
#                 for term in terms:
#                     for rate in rates:
#                         for battery_capex_per_kWh in battery_costs:
#                             for central_battery_capacity_kWh in central_battery_capacities[site]:
#                                 for central_battery_strategy in battery_strategies:
#                                     s += 1
#                                     df.loc[s,'pv_filename'] = pv_base + site+ '_' + pv +'.csv'
#                                     df.loc[s,'load_folder'] = lf
#                                     df.loc[s,'arrangement'] = arr
#                                     df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
#                                     df.loc[s,'cp'] = tariff
#                                     df.loc[s,'all_residents'] = tariff
#                                     df.loc[s,'parent'] = parent
#                                     df.loc[s,'network_tariff'] = net_tariff
#                                     df.loc[s,'en_capex_id'] = capex
#                                     df.loc[s,'a_term'] = term
#                                     df.loc[s,'a_rate'] = rate
#                                     df.loc[s,'central_battery_id'] = central_battery_id
#                                     df.loc[s,'central_battery_strategy'] = central_battery_strategy
#                                     df.loc[s,'central_battery_capacity_kWh'] = central_battery_capacity_kWh
#                                     df.loc[s,'battery_capex_per_kWh'] = battery_capex_per_kWh
    

# #     for arr in btm_s:
# #         parent ='TIDNULL'
# #         for pv in pvs[site]:     
# #             for tariff in btms_tariffs:
# #                 for capex in btms_capexes:                
# #                     for term in terms:
# #                         for rate in rates:
# #                             s += 1
# #                             df.loc[s,'pv_filename'] = pv_base + site + '_' + pv +'.csv'
# #                             df.loc[s,'load_folder'] = lf
# #                             df.loc[s,'arrangement'] = arr
# #                             df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
# #                             df.loc[s,'cp'] = tariff
# #                             df.loc[s,'all_residents'] = tariff
# #                             df.loc[s,'parent'] = 'TIDNULL'
# #                             df.loc[s,'network_tariff'] = net_tariff
# #                             df.loc[s,'en_capex_id'] = capex
# #                             df.loc[s,'a_term'] = term
# #                             df.loc[s,'a_rate'] = rate

# #     for arr in btm_p:
# #             parent ='TIDNULL'
# #             for pv in pvs[site]:     
# #                 for tariff in btmp_tariffs:
# #                     for capex in btmp_capexes:
# #                         for term in terms:
# #                             for rate in rates:
# #                                 s += 1
# #                                 df.loc[s,'pv_filename'] = pv_base + site + '_' + pv +'.csv'
# #                                 df.loc[s,'load_folder'] = lf
# #                                 df.loc[s,'arrangement'] = arr
# #                                 df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
# #                                 df.loc[s,'cp'] = tariff
# #                                 df.loc[s,'all_residents'] = tariff
# #                                 df.loc[s,'parent'] = 'TIDNULL'
# #                                 df.loc[s,'network_tariff'] = net_tariff
# #                                 df.loc[s,'en_capex_id'] = capex
# #                                 df.loc[s,'a_term'] = term
# #                                 df.loc[s,'a_rate'] = rate
                        
                        
# #     for arr in btm_i:
# #         parent ='TIDNULL'
# #         for pv in pvs[site]:     
# #             for tariff in btmi_tariffs:
# #                 capex ='' 
# #                 for term in terms:
# #                     for rate in rates:
# #                         s += 1
# #                         df.loc[s,'pv_filename'] = pv_base + site + '_' + pv +'.csv'
# #                         df.loc[s,'load_folder'] = lf
# #                         df.loc[s,'arrangement'] = arr
# #                         df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv + '_i'  # NB different capex for `btm_i`
# #                         df.loc[s,'cp'] = tariff
# #                         df.loc[s,'all_residents'] = tariff
# #                         df.loc[s,'parent'] = parent
# #                         df.loc[s,'network_tariff'] = net_tariff
# #                         df.loc[s,'en_capex_id'] = capex
# #                         df.loc[s,'a_term'] = term
# #                         df.loc[s,'a_rate'] = rate                    


# #     arr= 'cp_only'
# #     parent ='TIDNULL'
# #     for pv in pvs[site]:     
# #         for tariff in btmi_tariffs:
# #             for term in terms:
# #                 for rate in rates:
# #                     s += 1
# #                     df.loc[s,'pv_filename'] = pv_base + site+ '_' + pv +'.csv'
# #                     df.loc[s,'load_folder'] = lf
# #                     df.loc[s,'arrangement'] = arr
# #                     df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
# #                     df.loc[s,'cp'] = tariff
# #                     df.loc[s,'all_residents'] = tariff
# #                     df.loc[s,'parent'] = 'TIDNULL'
# #                     df.loc[s,'network_tariff'] = net_tariff
# #                     df.loc[s,'en_capex_id'] = ''
# #                     df.loc[s,'a_term'] = term
# #                     df.loc[s,'a_rate'] = rate


#     um.df_to_csv(df,file)
#     print ("****Saved ", file)
# print("****completed****")




In [3]:
battery_strategies= ['ed1700_cmax',
'ed1700_c25',
'ed1700_c10',
'ed1730_cmax',
'ed1730_c25',
'ed1730_c10',
'ed1800_s_cmax',
'ed1700_s_cmax',
'ed1800_s_c25',
'ed1700_s_c25',
'ch_ed1700_s_cmax',
'ch_ed1600_s_cmax']


In [4]:
# ------------------
# financial analysis finance 2
# ------------------
studybase = 'finance2'

# All batteries are scaleable:
# ---------------------------
central_battery_id = 'pw_scale'

# SIMPLIFICATIONs:
sites =['J', 'I', 'H', 'G','F']
en_capexes = ['capex_med']
terms=[20]
parent_retails =   ['TOU12']
parents = {'EA310': ['EA310_' + t for t in parent_retails],
           'EA305': ['EA305_' + t for t in parent_retails]}
battery_strategies = ['ed1700_s_cmax',
                      'ch_ed1700_s_cmax',
                      'ch_ed1600_s_cmax']
central_battery_capacities= dict(zip(sites, [[n*num_occs[site] for n in [1,2,3]] for site in sites]))              

# Separate '.csv' for each site:
for site in sites:
    net_tariff = net_tariffs[site]
    fn = 'study_' + studybase + '_' + site  +'.csv'
    file = os.path.join(i_path,fn)
    df = pd.DataFrame()
    df.index.name='scenario'
    lf = 'sh_t_all_site_'+site
    
    s=-1 # To start at 0 for bau
    
    arr ='bau'
    for term in terms:
        for rate in rates:
            s += 1
            df.loc[s,'pv_filename'] = ''
            df.loc[s,'load_folder'] = lf
            df.loc[s,'arrangement'] = arr
            df.loc[s,'pv_cap_id'] = ''
            df.loc[s,'cp'] = bau_tariff
            df.loc[s,'all_residents'] = bau_tariff
            df.loc[s,'parent'] = 'TIDNULL'
            df.loc[s,'network_tariff'] = net_tariff
            df.loc[s,'en_capex_id'] = ''
            df.loc[s,'a_term'] = term
            df.loc[s,'a_rate'] = rate
    
                
    arr ='en'
    tariff = 'TIDNULL'    
    for parent in parents[net_tariff]:
        for capex in en_capexes:
            for term in terms:
                for rate in rates:
                    for battery_capex_per_kWh in battery_costs:
                        for central_battery_capacity_kWh in central_battery_capacities[site]:
                            for central_battery_strategy in battery_strategies:
                                s += 1
                                df.loc[s,'pv_filename'] = ''
                                df.loc[s,'load_folder'] = lf
                                df.loc[s,'arrangement'] = arr
                                df.loc[s,'pv_cap_id'] = ''
                                df.loc[s,'cp'] = tariff
                                df.loc[s,'all_residents'] = tariff
                                df.loc[s,'parent'] = parent
                                df.loc[s,'network_tariff'] = net_tariff
                                df.loc[s,'en_capex_id'] = capex
                                df.loc[s,'a_term'] = term
                                df.loc[s,'a_rate'] = rate
                                df.loc[s,'central_battery_id'] = central_battery_id
                                df.loc[s,'central_battery_strategy'] = central_battery_strategy
                                df.loc[s,'central_battery_capacity_kWh'] = central_battery_capacity_kWh
                                df.loc[s,'battery_capex_per_kWh'] = battery_capex_per_kWh
     
    arr ='en'  # with no battery
    tariff = 'TIDNULL'
    for capex in en_capexes:
        for term in terms:
            for rate in rates:
                s += 1
                df.loc[s,'pv_filename'] = ''
                df.loc[s,'load_folder'] = lf
                df.loc[s,'arrangement'] = arr
                df.loc[s,'pv_cap_id'] = ''
                df.loc[s,'cp'] = tariff
                df.loc[s,'all_residents'] = tariff
                df.loc[s,'parent'] = parent
                df.loc[s,'network_tariff'] = net_tariff
                df.loc[s,'en_capex_id'] = capex
                df.loc[s,'a_term'] = term
                df.loc[s,'a_rate'] = rate
                df.loc[s,'central_battery_id'] = ''
                df.loc[s,'central_battery_strategy'] = ''
                df.loc[s,'central_battery_capacity_kWh'] = 0               
                df.loc[s,'battery_capex_per_kWh'] = 0
                df.loc[s,'output_types'] = 'log_timeseries_brief'

    
    arr ='en_pv' # with no battery
    tariff = 'TIDNULL'
    for pv in pvs[site]:
        for parent in parents[net_tariff]:           
            for capex in en_capexes:
                for term in terms:
                    for rate in rates:
                        s += 1
                        df.loc[s,'pv_filename'] = pv_base + site+ '_' + pv +'.csv'
                        df.loc[s,'load_folder'] = lf
                        df.loc[s,'arrangement'] = arr
                        df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
                        df.loc[s,'cp'] = tariff
                        df.loc[s,'all_residents'] = tariff
                        df.loc[s,'parent'] = parent
                        df.loc[s,'network_tariff'] = net_tariff
                        df.loc[s,'en_capex_id'] = capex
                        df.loc[s,'a_term'] = term
                        df.loc[s,'a_rate'] = rate
                        df.loc[s,'central_battery_id'] = ''
                        df.loc[s,'central_battery_strategy'] = ''
                               
                    
    arr ='en_pv'
    tariff = 'TIDNULL'
    for pv in pvs[site]:
        for parent in parents[net_tariff]:           
            for capex in en_capexes:
                for term in terms:
                    for rate in rates:
                        for battery_capex_per_kWh in battery_costs:
                            for central_battery_capacity_kWh in central_battery_capacities[site]:
                                for central_battery_strategy in battery_strategies:
                                    s += 1
                                    df.loc[s,'pv_filename'] = pv_base + site + '_' + pv +'.csv'
                                    df.loc[s,'load_folder'] = lf
                                    df.loc[s,'arrangement'] = arr
                                    df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
                                    df.loc[s,'cp'] = tariff
                                    df.loc[s,'all_residents'] = tariff
                                    df.loc[s,'parent'] = parent
                                    df.loc[s,'network_tariff'] = net_tariff
                                    df.loc[s,'en_capex_id'] = capex
                                    df.loc[s,'a_term'] = term
                                    df.loc[s,'a_rate'] = rate
                                    df.loc[s,'central_battery_id'] = central_battery_id
                                    df.loc[s,'central_battery_strategy'] = central_battery_strategy
                                    df.loc[s,'central_battery_capacity_kWh'] = central_battery_capacity_kWh
                                    df.loc[s,'battery_capex_per_kWh'] = battery_capex_per_kWh
    

#     for arr in btm_s:
#         parent ='TIDNULL'
#         for pv in pvs[site]:     
#             for tariff in btms_tariffs:
#                 for capex in btms_capexes:                
#                     for term in terms:
#                         for rate in rates:
#                             s += 1
#                             df.loc[s,'pv_filename'] = pv_base + site + '_' + pv +'.csv'
#                             df.loc[s,'load_folder'] = lf
#                             df.loc[s,'arrangement'] = arr
#                             df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
#                             df.loc[s,'cp'] = tariff
#                             df.loc[s,'all_residents'] = tariff
#                             df.loc[s,'parent'] = 'TIDNULL'
#                             df.loc[s,'network_tariff'] = net_tariff
#                             df.loc[s,'en_capex_id'] = capex
#                             df.loc[s,'a_term'] = term
#                             df.loc[s,'a_rate'] = rate

#     for arr in btm_p:
#             parent ='TIDNULL'
#             for pv in pvs[site]:     
#                 for tariff in btmp_tariffs:
#                     for capex in btmp_capexes:
#                         for term in terms:
#                             for rate in rates:
#                                 s += 1
#                                 df.loc[s,'pv_filename'] = pv_base + site + '_' + pv +'.csv'
#                                 df.loc[s,'load_folder'] = lf
#                                 df.loc[s,'arrangement'] = arr
#                                 df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
#                                 df.loc[s,'cp'] = tariff
#                                 df.loc[s,'all_residents'] = tariff
#                                 df.loc[s,'parent'] = 'TIDNULL'
#                                 df.loc[s,'network_tariff'] = net_tariff
#                                 df.loc[s,'en_capex_id'] = capex
#                                 df.loc[s,'a_term'] = term
#                                 df.loc[s,'a_rate'] = rate
                        
                        
#     for arr in btm_i:
#         parent ='TIDNULL'
#         for pv in pvs[site]:     
#             for tariff in btmi_tariffs:
#                 capex ='' 
#                 for term in terms:
#                     for rate in rates:
#                         s += 1
#                         df.loc[s,'pv_filename'] = pv_base + site + '_' + pv +'.csv'
#                         df.loc[s,'load_folder'] = lf
#                         df.loc[s,'arrangement'] = arr
#                         df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv + '_i'  # NB different capex for 'btm_i'
#                         df.loc[s,'cp'] = tariff
#                         df.loc[s,'all_residents'] = tariff
#                         df.loc[s,'parent'] = parent
#                         df.loc[s,'network_tariff'] = net_tariff
#                         df.loc[s,'en_capex_id'] = capex
#                         df.loc[s,'a_term'] = term
#                         df.loc[s,'a_rate'] = rate                    


#     arr= 'cp_only'
#     parent ='TIDNULL'
#     for pv in pvs[site]:     
#         for tariff in btmi_tariffs:
#             for term in terms:
#                 for rate in rates:
#                     s += 1
#                     df.loc[s,'pv_filename'] = pv_base + site+ '_' + pv +'.csv'
#                     df.loc[s,'load_folder'] = lf
#                     df.loc[s,'arrangement'] = arr
#                     df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
#                     df.loc[s,'cp'] = tariff
#                     df.loc[s,'all_residents'] = tariff
#                     df.loc[s,'parent'] = 'TIDNULL'
#                     df.loc[s,'network_tariff'] = net_tariff
#                     df.loc[s,'en_capex_id'] = ''
#                     df.loc[s,'a_term'] = term
#                     df.loc[s,'a_rate'] = rate


    um.df_to_csv(df,file)
    print ("****Saved ", file)
print("****completed****")




****Saved  C:\Users\z5044992\Documents\MainDATA\DATA_EN_4\studies\EN2_bat2\inputs\study_finance2_J.csv
****Saved  C:\Users\z5044992\Documents\MainDATA\DATA_EN_4\studies\EN2_bat2\inputs\study_finance2_I.csv
****Saved  C:\Users\z5044992\Documents\MainDATA\DATA_EN_4\studies\EN2_bat2\inputs\study_finance2_H.csv
****Saved  C:\Users\z5044992\Documents\MainDATA\DATA_EN_4\studies\EN2_bat2\inputs\study_finance2_G.csv
****Saved  C:\Users\z5044992\Documents\MainDATA\DATA_EN_4\studies\EN2_bat2\inputs\study_finance2_F.csv
****completed****


In [6]:
# ------------------
#  energy and financial analysis for cp_only
# ------------------
studybase = 'cponly'

# All batteries are scaleable:
# ---------------------------
central_battery_id = 'pw_scale'

# SIMPLIFICATIONs:
sites =['J', 'I', 'H', 'G','F']
en_capexes = ['capex_med']
terms=[20]
battery_strategies = ['ed1700_s_cmax',
                      'ch_ed1700_s_cmax',
                      'ch_ed1600_s_cmax']
cp_battery_capacities = dict(zip(sites, [[n*num_occs[site] for n in [1,2,3,4]] for site in sites]))              
parent ='TIDNULL'
tariff = 'TIDNULL'
cp_battery_id = 'pw_scale'  
all_battery_id = ''
all_battery_capacity_kWh =0
all_battery_strategy = ''
    
    
# Separate '.csv' for each site:
for site in sites:
    net_tariff = net_tariffs[site]
    fn = 'study_' + studybase + '_' + site  +'.csv'
    file = os.path.join(i_path,fn)
    df = pd.DataFrame()
    df.index.name='scenario'
    lf = 'sh_t_all_site_'+site
    
    s=-1 # To start at 0 for bau
    
    arr ='bau'
    for term in terms:
        for rate in rates:
            s += 1
            df.loc[s,'pv_filename'] = ''
            df.loc[s,'load_folder'] = lf
            df.loc[s,'arrangement'] = arr
            df.loc[s,'pv_cap_id'] = ''
            df.loc[s,'cp'] = bau_tariff
            df.loc[s,'all_residents'] = bau_tariff
            df.loc[s,'parent'] = 'TIDNULL'
            df.loc[s,'network_tariff'] = net_tariff
            df.loc[s,'en_capex_id'] = ''
            df.loc[s,'a_term'] = term
            df.loc[s,'a_rate'] = rate
    capex='capex_med'
    arr ='bau_bat' # No PV, battery for cp only   
    for tariff in btmi_tariffs:
        for term in terms:
            for rate in rates:
                for battery_capex_per_kWh in battery_costs:
                    for cp_battery_capacity_kWh in cp_battery_capacities[site]:     
                        for cp_battery_strategy in battery_strategies:

                            s += 1
                            df.loc[s,'pv_filename'] = ''
                            df.loc[s,'load_folder'] = lf
                            df.loc[s,'arrangement'] = arr
                            df.loc[s,'pv_cap_id'] = ''
                            df.loc[s,'cp'] = tariff
                            df.loc[s,'all_residents'] = tariff
                            df.loc[s,'parent'] = parent
                            df.loc[s,'network_tariff'] = net_tariff
                            df.loc[s,'en_capex_id'] = capex
                            df.loc[s,'a_term'] = term
                            df.loc[s,'a_rate'] = rate                    
                            df.loc[s,'cp_battery_id'] = cp_battery_id
                            df.loc[s,'cp_battery_strategy'] = cp_battery_strategy
                            df.loc[s,'cp_battery_capacity_kWh'] = cp_battery_capacity_kWh    
                            df.loc[s,'all_battery_id'] = all_battery_id
                            df.loc[s,'all_battery_strategy'] = all_battery_strategy
                            df.loc[s,'all_battery_capacity_kWh'] = all_battery_capacity_kWh    
                            df.loc[s,'battery_capex_per_kWh'] = battery_capex_per_kWh
                            df.loc[s,'output_types'] = 'log_timeseries_brief'

    arr= 'cp_only'
    parent ='TIDNULL'
    for pv in pvs[site]:     
        for tariff in btmi_tariffs:
            for term in terms:
                for rate in rates:
                    for battery_capex_per_kWh in battery_costs:
                        for cp_battery_capacity_kWh in cp_battery_capacities[site]:     
                            for cp_battery_strategy in battery_strategies:
                                s += 1
                                df.loc[s,'pv_filename'] = pv_base + site+ '_' + pv +'.csv'
                                df.loc[s,'load_folder'] = lf
                                df.loc[s,'arrangement'] = arr
                                df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv   
                                df.loc[s,'cp'] = tariff
                                df.loc[s,'all_residents'] = tariff
                                df.loc[s,'parent'] = parent
                                df.loc[s,'network_tariff'] = net_tariff
                                df.loc[s,'en_capex_id'] = capex
                                df.loc[s,'a_term'] = term
                                df.loc[s,'a_rate'] = rate                    
                                df.loc[s,'cp_battery_id'] = cp_battery_id
                                df.loc[s,'cp_battery_strategy'] = cp_battery_strategy
                                df.loc[s,'cp_battery_capacity_kWh'] = cp_battery_capacity_kWh    
                                df.loc[s,'all_battery_id'] = all_battery_id
                                df.loc[s,'all_battery_strategy'] = all_battery_strategy
                                df.loc[s,'all_battery_capacity_kWh'] = all_battery_capacity_kWh    
                                df.loc[s,'battery_capex_per_kWh'] = battery_capex_per_kWh
                                df.loc[s,'output_types'] = 'log_timeseries_brief'

    arr= 'cp_only' # No battery
    battery_capex_per_kWh =''
    cp_battery_capacity_kWh =0
    cp_battery_strategy =''
    for pv in pvs[site]:     
        for tariff in btmi_tariffs:
            for term in terms:
                for rate in rates:
                    
                    s += 1
                    df.loc[s,'pv_filename'] = pv_base + site+ '_' + pv +'.csv'
                    df.loc[s,'load_folder'] = lf
                    df.loc[s,'arrangement'] = arr
                    df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv   
                    df.loc[s,'cp'] = tariff
                    df.loc[s,'all_residents'] = tariff
                    df.loc[s,'parent'] = parent
                    df.loc[s,'network_tariff'] = net_tariff
                    df.loc[s,'en_capex_id'] = capex
                    df.loc[s,'a_term'] = term
                    df.loc[s,'a_rate'] = rate                    
                    df.loc[s,'cp_battery_id'] = cp_battery_id
                    df.loc[s,'cp_battery_strategy'] = cp_battery_strategy
                    df.loc[s,'cp_battery_capacity_kWh'] = cp_battery_capacity_kWh    
                    df.loc[s,'all_battery_id'] = all_battery_id
                    df.loc[s,'all_battery_strategy'] = all_battery_strategy
                    df.loc[s,'all_battery_capacity_kWh'] = all_battery_capacity_kWh    
                    df.loc[s,'battery_capex_per_kWh'] = battery_capex_per_kWh
                    df.loc[s,'output_types'] = 'log_timeseries_brief'
#     arr ='en'
#     tariff = 'TIDNULL'    
#     for parent in parents[net_tariff]:
#         for capex in en_capexes:
#             for term in terms:
#                 for rate in rates:
#                     for battery_capex_per_kWh in battery_costs:
#                         for central_battery_capacity_kWh in central_battery_capacities[site]:
#                             for central_battery_strategy in battery_strategies:
#                                 s += 1
#                                 df.loc[s,'pv_filename'] = ''
#                                 df.loc[s,'load_folder'] = lf
#                                 df.loc[s,'arrangement'] = arr
#                                 df.loc[s,'pv_cap_id'] = ''
#                                 df.loc[s,'cp'] = tariff
#                                 df.loc[s,'all_residents'] = tariff
#                                 df.loc[s,'parent'] = parent
#                                 df.loc[s,'network_tariff'] = net_tariff
#                                 df.loc[s,'en_capex_id'] = capex
#                                 df.loc[s,'a_term'] = term
#                                 df.loc[s,'a_rate'] = rate
#                                 df.loc[s,'central_battery_id'] = central_battery_id
#                                 df.loc[s,'central_battery_strategy'] = central_battery_strategy
#                                 df.loc[s,'central_battery_capacity_kWh'] = central_battery_capacity_kWh
#                                 df.loc[s,'battery_capex_per_kWh'] = battery_capex_per_kWh
     
#     arr ='en'  # with no battery
#     tariff = 'TIDNULL'
#     for capex in en_capexes:
#         for term in terms:
#             for rate in rates:
#                 s += 1
#                 df.loc[s,'pv_filename'] = ''
#                 df.loc[s,'load_folder'] = lf
#                 df.loc[s,'arrangement'] = arr
#                 df.loc[s,'pv_cap_id'] = ''
#                 df.loc[s,'cp'] = tariff
#                 df.loc[s,'all_residents'] = tariff
#                 df.loc[s,'parent'] = parent
#                 df.loc[s,'network_tariff'] = net_tariff
#                 df.loc[s,'en_capex_id'] = capex
#                 df.loc[s,'a_term'] = term
#                 df.loc[s,'a_rate'] = rate
#                 df.loc[s,'central_battery_id'] = ''
#                 df.loc[s,'central_battery_strategy'] = ''
#                 df.loc[s,'central_battery_capacity_kWh'] = 0               
#                 df.loc[s,'battery_capex_per_kWh'] = 0
#                 df.loc[s,'output_types'] = 'log_timeseries_brief'

    
#     arr ='en_pv' # with no battery
#     tariff = 'TIDNULL'
#     for pv in pvs[site]:
#         for parent in parents[net_tariff]:           
#             for capex in en_capexes:
#                 for term in terms:
#                     for rate in rates:
#                         s += 1
#                         df.loc[s,'pv_filename'] = pv_base + site+ '_' + pv +'.csv'
#                         df.loc[s,'load_folder'] = lf
#                         df.loc[s,'arrangement'] = arr
#                         df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
#                         df.loc[s,'cp'] = tariff
#                         df.loc[s,'all_residents'] = tariff
#                         df.loc[s,'parent'] = parent
#                         df.loc[s,'network_tariff'] = net_tariff
#                         df.loc[s,'en_capex_id'] = capex
#                         df.loc[s,'a_term'] = term
#                         df.loc[s,'a_rate'] = rate
#                         df.loc[s,'central_battery_id'] = ''
#                         df.loc[s,'central_battery_strategy'] = ''
                               
                    
#     arr ='en_pv'
#     tariff = 'TIDNULL'
#     for pv in pvs[site]:
#         for parent in parents[net_tariff]:           
#             for capex in en_capexes:
#                 for term in terms:
#                     for rate in rates:
#                         for battery_capex_per_kWh in battery_costs:
#                             for central_battery_capacity_kWh in central_battery_capacities[site]:
#                                 for central_battery_strategy in battery_strategies:
#                                     s += 1
#                                     df.loc[s,'pv_filename'] = pv_base + site+ '_' + pv +'.csv'
#                                     df.loc[s,'load_folder'] = lf
#                                     df.loc[s,'arrangement'] = arr
#                                     df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
#                                     df.loc[s,'cp'] = tariff
#                                     df.loc[s,'all_residents'] = tariff
#                                     df.loc[s,'parent'] = parent
#                                     df.loc[s,'network_tariff'] = net_tariff
#                                     df.loc[s,'en_capex_id'] = capex
#                                     df.loc[s,'a_term'] = term
#                                     df.loc[s,'a_rate'] = rate
#                                     df.loc[s,'central_battery_id'] = central_battery_id
#                                     df.loc[s,'central_battery_strategy'] = central_battery_strategy
#                                     df.loc[s,'central_battery_capacity_kWh'] = central_battery_capacity_kWh
#                                     df.loc[s,'battery_capex_per_kWh'] = battery_capex_per_kWh
    

#     for arr in btm_s:
#         parent ='TIDNULL'
#         for pv in pvs[site]:     
#             for tariff in btms_tariffs:
#                 for capex in btms_capexes:                
#                     for term in terms:
#                         for rate in rates:
#                             s += 1
#                             df.loc[s,'pv_filename'] = pv_base + site + '_' + pv +'.csv'
#                             df.loc[s,'load_folder'] = lf
#                             df.loc[s,'arrangement'] = arr
#                             df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
#                             df.loc[s,'cp'] = tariff
#                             df.loc[s,'all_residents'] = tariff
#                             df.loc[s,'parent'] = 'TIDNULL'
#                             df.loc[s,'network_tariff'] = net_tariff
#                             df.loc[s,'en_capex_id'] = capex
#                             df.loc[s,'a_term'] = term
#                             df.loc[s,'a_rate'] = rate

#     for arr in btm_p:
#             parent ='TIDNULL'
#             for pv in pvs[site]:     
#                 for tariff in btmp_tariffs:
#                     for capex in btmp_capexes:
#                         for term in terms:
#                             for rate in rates:
#                                 s += 1
#                                 df.loc[s,'pv_filename'] = pv_base + site + '_' + pv +'.csv'
#                                 df.loc[s,'load_folder'] = lf
#                                 df.loc[s,'arrangement'] = arr
#                                 df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
#                                 df.loc[s,'cp'] = tariff
#                                 df.loc[s,'all_residents'] = tariff
#                                 df.loc[s,'parent'] = 'TIDNULL'
#                                 df.loc[s,'network_tariff'] = net_tariff
#                                 df.loc[s,'en_capex_id'] = capex
#                                 df.loc[s,'a_term'] = term
#                                 df.loc[s,'a_rate'] = rate
                        
                        
#     for arr in btm_i:
#         parent ='TIDNULL'
#         for pv in pvs[site]:     
#             for tariff in btmi_tariffs:
#                 capex ='' 
#                 for term in terms:
#                     for rate in rates:
#                         s += 1
#                         df.loc[s,'pv_filename'] = pv_base + site + '_' + pv +'.csv'
#                         df.loc[s,'load_folder'] = lf
#                         df.loc[s,'arrangement'] = arr
#                         df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv + '_i'  # NB different capex for 'btm_i'
#                         df.loc[s,'cp'] = tariff
#                         df.loc[s,'all_residents'] = tariff
#                         df.loc[s,'parent'] = parent
#                         df.loc[s,'network_tariff'] = net_tariff
#                         df.loc[s,'en_capex_id'] = capex
#                         df.loc[s,'a_term'] = term
#                         df.loc[s,'a_rate'] = rate                    


#     arr= 'cp_only'
#     parent ='TIDNULL'
#     for pv in pvs[site]:     
#         for tariff in btmi_tariffs:
#             for term in terms:
#                 for rate in rates:
#                     s += 1
#                     df.loc[s,'pv_filename'] = pv_base + site+ '_' + pv +'.csv'
#                     df.loc[s,'load_folder'] = lf
#                     df.loc[s,'arrangement'] = arr
#                     df.loc[s,'pv_cap_id'] = 'site_'+site +'_'+ pv
#                     df.loc[s,'cp'] = tariff
#                     df.loc[s,'all_residents'] = tariff
#                     df.loc[s,'parent'] = 'TIDNULL'
#                     df.loc[s,'network_tariff'] = net_tariff
#                     df.loc[s,'en_capex_id'] = ''
#                     df.loc[s,'a_term'] = term
#                     df.loc[s,'a_rate'] = rate


    um.df_to_csv(df,file)
    print ("****Saved ", file)
print("****completed****")




****Saved  C:\Users\z5044992\Documents\MainDATA\DATA_EN_4\studies\EN2_bat2\inputs\study_cponly_J.csv
****Saved  C:\Users\z5044992\Documents\MainDATA\DATA_EN_4\studies\EN2_bat2\inputs\study_cponly_I.csv
****Saved  C:\Users\z5044992\Documents\MainDATA\DATA_EN_4\studies\EN2_bat2\inputs\study_cponly_H.csv
****Saved  C:\Users\z5044992\Documents\MainDATA\DATA_EN_4\studies\EN2_bat2\inputs\study_cponly_G.csv
****Saved  C:\Users\z5044992\Documents\MainDATA\DATA_EN_4\studies\EN2_bat2\inputs\study_cponly_F.csv
****completed****
